# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [18]:
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig

from azureml.train.hyperdrive.parameter_expressions import normal, choice

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import joblib

from train import clean_data

from sklearn.model_selection import train_test_split
import pandas as pd

In [19]:
CAPSTONE_EXPERIMENT_NAME_HYPER = 'exp-capstone-hyper'
CAPSTONE_DATASET_NAME = 'White Wine Data'
CAPSTONE_DATASET_DESCRIPTION = 'Wine data - does the wine taste good?'
CAPSTONE_TABULAR_WINE_DATA = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'
CAPSTONE_DEPLOYED_HYPER_MODEL_NAME = 'wine-taste-hyper'
CAPSTONE_DEPLOYED_HYPER_MODEL_PATH = './outputs/best_run_hyperdrive.pkl' 

TRAIN_DATA_DIR = 'train_norm_data'
TRAIN_DATA_FILE = 'train_norm.csv'
TRAIN_NORM_INFO_FILE = 'norm.csv'

cc_name = "CPU-CC-HYPER"  # CPU Compute Cluster
curated_env_name = 'AzureML-Tutorial'



## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [20]:
ws = Workspace.from_config()
experiment_name = CAPSTONE_EXPERIMENT_NAME_HYPER

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()




# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


# Use https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py as a reference
#
# Name the cluster GPUCluster
cluster_name = cc_name

# See if it already exists
# this ofcourse helps if you are reunning the notebook from teh start and do not
# need to recreate the compute cluster
try:
    compute_cluster = ComputeTarget(ws, cluster_name)
    # No exception thrown - Found it - use it below
except ComputeTargetException:
    # Did not find the compute target - will need to create one
    # Specify the compute cluster configuration first
    # See https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target
    # and after clicking on Dv2
    # see https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target

    # CPU cluster instance - from the Microsoft recommended options
    cluster_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_DS3_v2',
                                                            max_nodes=4,
                                                            description='CPU Compute Cluster created programatically')

    # Next, create the cluster
    compute_cluster = ComputeTarget.create(ws, cluster_name, cluster_config)

# We have a compute cluster - either newly created - or created earlier
# We may wait for the create operation to complete
compute_cluster.wait_for_completion(show_output=True)








Workspace name: quick-starts-ws-161266
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-161266
InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [21]:
# Create TabularDataset using TabularDatasetFactory

# Create the Azure ML dataset from the preferred source
# Note that thsi source is the same as used in train.py
# grab the data and create a dataset
# See if the dataset already exists - if so, skip the Dataset creation pieces

ds_name = CAPSTONE_DATASET_NAME
dsets = ws.datasets.keys()

if ds_name in dsets:
    # dataset exists
    train_ds = ws.datasets[ds_name]
else:
    # Data set not found. Must create it
    # This is the original white wine data - not normalized
    # We will normalize the data and then create a dataset that
    # then will continue to be used
    ds = TabularDatasetFactory.from_delimited_files(CAPSTONE_TABULAR_WINE_DATA, separator=';')
    
    X, y, norm_df = clean_data(ds.to_pandas_dataframe())
    
    # Split the data into train/test sets
    # Note that the training data would be used for AutoML; 
    # the test data is not put to use in this project
    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2)
    
    # Add x and y together
    # save the new df to disk as a csv
    # Upload to a datastore
    # load from datastore as an Azure TabularDataSet
    
    # Concat two pandas dataframes togethere
    train_data = pd.concat([X_train, y_train], axis=1)
    
    # From here on - X_train contains both input + output

    # save and reload the clean data so that Azure ML can use it
    # See https://stackoverflow.com/questions/60380154/upload-dataframe-as-dataset-in-azure-machine-learning
    
    # To be able to load to datastore - the data needs to be in a folder.
    # Thus first create the directory if it does not exist
    file_path = os.path.join(TRAIN_DATA_DIR, TRAIN_DATA_FILE)
    norm_file_path = os.path.join(TRAIN_DATA_DIR, TRAIN_NORM_INFO_FILE)
    if TRAIN_DATA_DIR not in os.listdir():
      os.mkdir(os.path.join('.', TRAIN_DATA_DIR))
    else:
      # delete contents of directory
      os.remove(file_path)
      os.remove(norm_file_path)
      
    # now save the training data to disk
    train_data.to_csv(file_path, index=False)
    norm_df.to_csv(norm_file_path)
    
    # upload the file to the default datastore
    datastore = ws.get_default_datastore()
    datastore.upload(src_dir=TRAIN_DATA_DIR, target_path=TRAIN_DATA_DIR, overwrite=True)

    # Now Create the training dataset 
    train_ds = TabularDatasetFactory.from_delimited_files(datastore.path(file_path))
    
    # Register the dataset so that on repeated runs, the data does not have to be fetched evey time
    train_ds = train_ds.register(workspace=ws, name=ds_name, description=CAPSTONE_DATASET_DESCRIPTION)
            
 

In [22]:

# Take a peek at the data by converting the same to a Pandas dataframe
train_df = train_ds.to_pandas_dataframe()
    
# print the data
train_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,2.186612,0.315085,3.270609,1.569104,-0.081122,-0.488506,1.238629,2.040392,-1.511694,3.418622,-1.474270,0
1,-0.538932,-0.478608,-0.034635,-1.033390,-0.081122,-1.164692,1.862187,-0.677847,0.938628,-0.436771,0.394706,0
2,1.357099,0.513509,0.130627,0.711464,-0.493060,-0.782500,-0.714401,0.181424,-0.452095,-1.137752,0.882265,0
3,-1.842453,1.902473,-1.935150,-1.003816,-0.447289,-1.429287,-1.514437,-1.072376,1.932001,0.439455,0.557225,0
4,-0.301928,-0.180973,1.865881,1.776120,0.330815,-0.194512,0.062105,1.361668,-0.518320,0.001342,-0.986711,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3913,0.764589,1.406414,-1.191470,0.100272,-0.218435,1.569454,0.273879,0.351941,0.276379,-1.050129,-0.417892,0
3914,1.475600,-1.569937,0.378521,-1.082680,-0.538831,0.040684,-1.232072,-1.259610,-2.571292,0.527077,1.207304,0
3915,2.068110,0.910356,3.022716,1.135355,-0.309976,-0.841299,0.462123,1.428538,-1.246794,2.104284,-0.986711,0
3916,-1.486947,0.414297,-0.034635,0.198851,-0.401518,-0.606103,-1.020297,-0.450491,0.408829,0.001342,0.475966,0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [26]:
# BanditPolicy is a type of EarlyTerminationPolicy
# More info can be found at 
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

# from train.py - we know that the parameters are --C and --max_iter
# train.py uses the scikit-learn LogisticRegression model

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": choice(0.001, 0.01, 0.1, 1, 10),
    "--max_iter": choice(50, 100, 200, 300, 400),
})


# We need to specify/provide an environment
# We could create our own environment with corresponding yaml file
# We may also provide an environment 'out-of-the-box' from one of the many environments provided

curated_env = Environment.get(workspace=ws, name=curated_env_name)

# SKLearn is deprecated - instead we specity the run configuration using ScriptRunConfig
# It is necessary to pass the directory, the python script, the compute cluster and the environment

#TODO: Create your estimator and hyperdrive config
# estimator = <your estimator here>


run_config = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=compute_cluster,
    environment=curated_env,
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    run_config=run_config,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
)

In [30]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(config=hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [32]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [31]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_20a8c4fc-da01-4c68-a175-f11bb857a01f
Web View: https://ml.azure.com/runs/HD_20a8c4fc-da01-4c68-a175-f11bb857a01f?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-161266/workspaces/quick-starts-ws-161266&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-10-18T00:48:04.417986][API][INFO]Experiment created<END>\n""<START>[2021-10-18T00:48:04.903100][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space<END>\n""<START>[2021-10-18T00:48:05.142241][GENERATOR][INFO]Successfully sampled '20' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_20a8c4fc-da01-4c68-a175-f11bb857a01f
Web View: https://ml.azure.com/runs/HD_20a8c4fc-da01-4c68-a175-f11bb857a01f?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-161266/workspaces/quick-starts-ws-161266&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get your best run and save the model from that run.


### YOUR CODE HERE ###
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

# Get the model hyperparameter values corresponding to the best run
print(f'********** {best_hyperdrive_run.get_details()["runDefinition"]["arguments"]}')
# print(best_hyperdrive_run.get_details())

# Get the accuracy corresponsding to the best run
print(f'********** Best HyperDrive run Accuracy: {best_hyperdrive_run.get_metrics()["Accuracy"]}')

In [ ]:
#TODO: Save the best model
# Save and download the model
print(f'The file name for the model is: {best_hyperdrive_run.get_file_names()[-1]}')

# download the file in the notebook context. In this case, under Users/**user**/outputs/
best_hyperdrive_run.download_file(name=best_hyperdrive_run.get_file_names()[-1], output_file_path=CAPSTONE_DEPLOYED_MODEL_PATH)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = best_hyperdrive_run.register_model(model_name=CAPSTONE_DEPLOYED_HYPER_MODEL_NAME, 
                                           model_path = CAPSTONE_DEPLOYED_HYPER_MODEL_PATH)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

